<a href="https://colab.research.google.com/github/itzpankajpanwar/FakeNewsPropagation/blob/main/GAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages.
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric
!pip install vaderSentiment

     |████████████████████████████████| 2.6MB 15.0MB/s 
     |████████████████████████████████| 1.5MB 25.0MB/s 
     |████████████████████████████████| 215kB 16.9MB/s 
     |████████████████████████████████| 235kB 47.3MB/s 
     |████████████████████████████████| 2.2MB 48.5MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 133kB 17.5MB/s 


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(2020) # seed for reproducible numbers

In [ ]:
!git clone https://github.com/itzpankajpanwar/FakeNewsPropagation.git

Cloning into 'FakeNewsPropagation'...
remote: Enumerating objects: 11383, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 11383 (delta 25), reused 0 (delta 0), pack-reused 11337
Receiving objects: 100% (11383/11383), 149.79 MiB | 24.65 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Checking out files: 100% (11295/11295), done.


In [ ]:
import sys
sys.path.insert(1,"/content/FakeNewsPropagation")
from  construct_sample_features import get_TPNF_dataset
sample_feature_arr = get_TPNF_dataset("data/features", "gossipcop", True, True, True,
                                        True, False, None, False)
print("\nsample array shape is : ",sample_feature_arr.shape)

(['Micro - Tree depth', 'Micro - No of nodes', 'Micro - Maximum out degree', 'No. of cascades with replies', 'Fraction of cascades with replies', 'Macro - Tree depth', 'Macro - No of nodes', 'Macro - Maximum out degree', 'Macro - No of cascades', "Macro - Max out degree node's level", 'No. of cascades with retweets', 'Fraction of cascades with retweets', 'No. of bot users retweeting', 'Fraction of bot user retweeting'], ['S10', 'S11', 'S12', 'S13', 'S14', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9'])
Feature group : Structural
3684
3684
(['Average time diff between adjacent replies', 'Time diff between first tweet posting node and first reply node', 'Time diff between first tweet posting news and last reply node', 'Average time between adjacent reply nodes in the deepest cascade', 'Time diff between first tweet posting news and last reply node in the deepest cascade', 'Average time diff between the adjacent retweet nodes in macro network', 'Time diff between first tweet and  m

In [ ]:
import torch
from torch_geometric.data import Data
import networkx as nx
import json
import os
from networkx.readwrite import json_graph
from load_dataset import get_dataset_sample_ids
import numpy as np


news_sourc = ["gossipcop"]

data_list = []
tt=0
for news_source in news_sourc:
    dataset_dir = "/content/FakeNewsPropagation/data/nx_network_data/nx_network_data"
    labels = ["fake","real"]

    
    for  news_label in labels:
        news_dataset_dir = "{}/{}_{}".format(dataset_dir, news_source, news_label)
        for sample_id in get_dataset_sample_ids(news_source, news_label, "/content/FakeNewsPropagation/data/sample_ids"):
            with open("{}/{}.json".format(news_dataset_dir, sample_id)) as file:
                new_graph = json_graph.tree_graph(json.load(file))
                i=0
                visited={}
                edge_list = new_graph.edges()
                edge_index = []
                for x , y in edge_list:
                  if x in visited:
                    if y in visited:
                      edge_index.append([visited[x],visited[y]])
                    else:
                       edge_index.append([visited[x],i])
                       visited[y]=i
                       i=i+1
                  else: 
                     if y in visited:
                      edge_index.append([i,visited[y]])
                      visited[x]=i
                      i=i+1
                     else:
                       edge_index.append([i,i+1])    
                       visited[x]=i
                       visited[y]=i+1
                       i=i+2           
                  #print(visited[x],"  ",visited[y],"\n")
                nodefeature=[]
                deg_centrality = nx.degree_centrality(new_graph)
                clos_centrality = nx.closeness_centrality(new_graph)
                page_centrality = nx.pagerank(new_graph,alpha=0.8)
                for x in deg_centrality:
                  trt=sample_feature_arr[tt]
                  at=np.array([])
                  at = np.append(at,deg_centrality[x])
                  at = np.append(at,clos_centrality[x])
                  at = np.append(at,page_centrality[x])
                  nodefeature.append(np.append(trt, at))
            
                tt+=1
              
                edge_index = torch.from_numpy(np.array(edge_index, dtype=np.long))
                x = torch.from_numpy(np.array(nodefeature, dtype=np.long))
                if news_label == "fake":
                  #for _ in range(nd):
                  y=0
                else:
                  #for _ in range(nd):
                  y=1
                data = Data(x=x, edge_index=edge_index.t().contiguous(),y=y)
                data_list.append(data)
print("length of data is : ",len(data_list))

length of data is :  7368


In [ ]:
from torch_geometric.data import Data, DataLoader
from torch_sparse import SparseTensor
loader = DataLoader(data_list)

#dataset = loader.dataset.shuffle()

train_size = int(0.8 * len(loader.dataset))
test_size = len(loader.dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(loader.dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, num_workers=0, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=0, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 32
Batch(batch=[10732], edge_index=[2, 10700], ptr=[33], x=[10732, 30], y=[32])

Step 2:
Number of graphs in the current batch: 32
Batch(batch=[4241], edge_index=[2, 4209], ptr=[33], x=[4241, 30], y=[32])

Step 3:
Number of graphs in the current batch: 32
Batch(batch=[11019], edge_index=[2, 10987], ptr=[33], x=[11019, 30], y=[32])

Step 4:
Number of graphs in the current batch: 32
Batch(batch=[6256], edge_index=[2, 6224], ptr=[33], x=[6256, 30], y=[32])

Step 5:
Number of graphs in the current batch: 32
Batch(batch=[4031], edge_index=[2, 3999], ptr=[33], x=[4031, 30], y=[32])

Step 6:
Number of graphs in the current batch: 32
Batch(batch=[8231], edge_index=[2, 8199], ptr=[33], x=[8231, 30], y=[32])

Step 7:
Number of graphs in the current batch: 32
Batch(batch=[3732], edge_index=[2, 3700], ptr=[33], x=[3732, 30], y=[32])

Step 8:
Number of graphs in the current batch: 32
Batch(batch=[5058], edge_index=[2, 5026], ptr=[33], x=[5058, 30], y=[

In [ ]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool


class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 8
        self.in_head = 8
        self.out_head = 1
        
        self.conv1 = GATConv(30, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, 2, concat=False,
                             heads=self.out_head, dropout=0.6)

    def forward(self, data):
        x, edge_index,batch = data.x, data.edge_index ,data.batch
        
        # Dropout before the GAT layer is used to avoid overfitting in small datasets like Cora.
        # One can skip them if the dataset is sufficiently large.
        
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        return F.log_softmax(x, dim=1)


In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GAT()
crit = torch.nn.CrossEntropyLoss()
#crite = F.nll_loss()
#data = loader[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)


def train():
    model.train()
    loss_all=0
    for data in iter(loader):
      data.x=data.x.float()
      for a in data.edge_index:
        a[0]=a[0].float()
        a[1]=a[1].float()
      optimizer.zero_grad()
      out = model(data)
      #print("\noutput  is : ",out)
      #print("\input shape   is : ",len(data.y))
      loss = crit(out, data.y)
      #print(loss,"\n")
      loss.backward()
      optimizer.step()


def test(loader,epoch):
     model.eval()

     correct =0
     for data in loader:  # Iterate in batches over the training/test dataset.
         data.x=data.x.float()
         for a in data.edge_index:
          a[0]=a[0].float()
          a[1]=a[1].float()
         out = model(data)  
         #print("output shape : ",out.shape)
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         #print("\nprediction size : ",pred,"         ",len(data.y))
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     #print("\n correct == ",correct,"       total = ",len(loader.dataset))
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1,101):
  train()
  train_acc = test(train_loader,epoch)
  test_acc = test(test_loader,epoch)
  print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.7f}, Test Acc: {test_acc:.7f}')

Epoch: 001, Train Acc: 0.5210383, Test Acc: 0.5583446
Epoch: 002, Train Acc: 0.5213777, Test Acc: 0.5610583
Epoch: 003, Train Acc: 0.5220563, Test Acc: 0.5637720
Epoch: 004, Train Acc: 0.5227350, Test Acc: 0.5664858
Epoch: 005, Train Acc: 0.5234136, Test Acc: 0.5691995
Epoch: 006, Train Acc: 0.5240923, Test Acc: 0.5719132
Epoch: 007, Train Acc: 0.5247710, Test Acc: 0.5746269
Epoch: 008, Train Acc: 0.5276552, Test Acc: 0.5820896
Epoch: 009, Train Acc: 0.5261283, Test Acc: 0.5800543
Epoch: 010, Train Acc: 0.5268069, Test Acc: 0.5827680
Epoch: 011, Train Acc: 0.5274856, Test Acc: 0.5854817
Epoch: 012, Train Acc: 0.5281642, Test Acc: 0.5881954
Epoch: 013, Train Acc: 0.5288429, Test Acc: 0.5909091
Epoch: 014, Train Acc: 0.5295215, Test Acc: 0.5936228
Epoch: 015, Train Acc: 0.5302002, Test Acc: 0.5963365
Epoch: 016, Train Acc: 0.5308789, Test Acc: 0.5990502
Epoch: 017, Train Acc: 0.5315575, Test Acc: 0.6017639
Epoch: 018, Train Acc: 0.5322362, Test Acc: 0.6044776
Epoch: 019, Train Acc: 0.532